In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Notebook Options on right, toggle Internet ON if PIP installing

import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV, cross_val_score

print("TensorFlow v" + tf.__version__)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

TensorFlow v2.16.1
/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [ ]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
train_data.head(10)

In [ ]:
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
test_data.head(20)


In [ ]:
all_data = pd.concat([train_data, test_data])

In [ ]:
train_data.isna().sum()

In [ ]:
train_data.dtypes

# **Find the top features**

**numeric correlation**

In [ ]:
features_numeric = ["Survived", "Fare", "Age"]
# 计算数值特征与目标变量的皮尔逊相关系数
correlation_matrix = train_data[features_numeric].corr()
corr_with_target = correlation_matrix['Survived'].sort_values(ascending=False)

print("数值特征与Survived的相关性：")
print(corr_with_target)

**category correlation**

In [ ]:
from sklearn.feature_selection import chi2
import pandas as pd

# 假设 all_data 是你的数据集，包含 Survived 和类别特征
# 提取类别型特征和目标变量
categorical_features =["Survived", "Pclass", "Sex", "SibSp", "Parch","Name","Ticket","Cabin","Embarked"]  # 示例类别特征
X_categorical = train_data[["Pclass", "Sex", "SibSp", "Parch","Name","Ticket","Cabin","Embarked"]].astype("str")
y = train_data['Survived']

# 将类别型特征转换为数值类型（编码）
X_categorical_encoded = pd.get_dummies(X_categorical)

# 使用卡方检验来计算每个类别特征与目标变量的相关性
chi_scores = chi2(X_categorical_encoded, y)

# 输出卡方统计量和 p 值
chi_scores_df = pd.DataFrame({
    'Feature': X_categorical_encoded.columns,
    'Chi2 Score': chi_scores[0],
    'P-value': chi_scores[1]
}).sort_values(by='Chi2 Score', ascending=False)

print("类别特征与Survived的相关性：")
print(chi_scores_df)




**Age -0.077221  Sex p-value 6.210585e-39 which is much smaller than 0.05**

**so I supposed that feature Sex may have more dedication to target-y**

# **Update Sex to binary value**

In [ ]:
#进行one-hot编码,对于小类别，例如Sex
all_data['Sex'] = all_data['Sex'].map({'male': 0, 'female': 1})

# **plot the heat map of all features**

In [ ]:
features = ["Survived", "Pclass", "Sex", "SibSp", "Parch", "Fare", "Age"]#corr()只能识别数值型特征


age_data = all_data[features]

corr = age_data.corr()

plt.figure(figsize=(7, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix Heatmap')
plt.show()

**The highest correlated feature with Age is the Pclass. I fill the missing values by the median of the age by Pclass.just a kind of like the way of Target-encoder**

In [ ]:
median_ages = all_data.groupby("Pclass")["Age"].transform('median')
all_data["Age"] = all_data["Age"].fillna(median_ages)

# **Cabin**
I replace this column with the following two features

Deck: The first letter in the cabin number. Set to "U" if Cabin is missing.

CabinMissing: 1 if cabin is missing, 0 otherwise.

In [ ]:
all_data["Deck"] = all_data["Cabin"].str[0].fillna("U")
all_data["CabinMissing"] = all_data["Cabin"].isna().astype("int")

In [ ]:
survival_counts = all_data[:891].groupby(['Deck', 'Survived']).size().unstack()

survival_counts.plot(kind='bar', stacked=False, figsize=(10, 6), color=['salmon', 'skyblue'])

plt.title('Survival Counts by Deck')
plt.xlabel('Deck')
plt.ylabel('Number of People')
plt.xticks(rotation=0)
plt.legend(title='Survived', labels=['Did not Survive', 'Survived'])
plt.grid(True)

plt.show()

# **Embarked**

I fill the missing values with the most common value for similar rows. 

There are only 2 missing values, so I set them to the mode embarked value of the passengers who are also female and on Deck "B".

In [ ]:
all_data[all_data["Embarked"].isna()]

In [ ]:
# 自定义函数：计算众数或返回 NaN
def mode_or_NaN(x):
    mode = x.mode()#x为各组下Embarked,选取各组下Embarked的众数，即x.mode()
    return mode[0] if not mode.empty else np.nan#mode()在python中，mode()返回series数据类型包含索引和值，用mode[i]来取
    
all_data["Embarked"] = all_data.groupby(["Sex","Deck"])["Embarked"].transform(lambda x:x.fillna(mode_or_NaN(x)))


# **Fare**
**I fill the missing fare values with the mean fare value for the highest correlated feature, which is the Pclass.**

In [ ]:
all_data[all_data.Fare.isna()]

In [ ]:
all_data["Fare"] = all_data.groupby("Pclass")["Fare"].transform(lambda x:x.fillna(x.mean()))

# **New Features**

**TicketCounts**

**This is set equal to the number of passengers with the same Ticket value. This might imply that they are part of the same group or family.**

In [ ]:
all_data["TicketCounts"] = all_data.groupby("Ticket")["Ticket"].transform('count')

**FamilySize**

**This is a new feature that is equal to SibSp + Parch + 1 (the passenger themself).**

**The family size might be another indicator for the survival probability of the passenger**

In [ ]:
all_data["FamilySize"] = all_data["SibSp"] + all_data["Parch"] + 1

**IsAlone**

**Set to 1 if the passenger is traveling alone, 0 otherwise. This is based on the FamilySize feature's value.**

In [ ]:
all_data["IsAlone"] = (all_data["FamilySize"] == 1).astype(int)

**AgeBin**

**This is an ordinal feature that groups the passengers by what range their age falls in.**

In [ ]:
all_data["Age"].describe()

In [ ]:
bins = [x*10 for x in range(10)]
labels = range(1,len(bins))
all_data["AgeBin"] = pd.cut(all_data["Age"],bins=bins,labels=labels,right=False)

**Title**

**It seems to carry potential information such as their gender (Mr. for male, Mrs. for female) and their age (Master. is given to boys).**

In [ ]:
all_data["Title"] = all_data["Name"].apply(lambda x:x.split(',')[1].split()[0])
all_data["Title"].unique()

In [ ]:
plt.figure(figsize=(7, 6))
all_data["Title"].value_counts().plot(kind='bar', color='skyblue')
plt.show()

In [ ]:
Match_list = ['the']
all_data[all_data["Name"].apply(lambda x:x.split(',')[1].split()[0]).isin(Match_list)]

In [ ]:
Match_list = ["Jonkheer.", "Dona.", "Mlle.", "Mme.", "Don."]
all_data[all_data["Name"].apply(lambda x:x.split(',')[1].split()[0]).isin(Match_list)]

**"the" is for PassengerId 760. It's followed by the term "Countess".**

**Jonkheer., Dona., Mlle., MMe. and Don. seem to be the names of passengers.**

**according to the bar chart,we can get the frequencies'columns :Mr,Mrs/Miss,Master,Officer/Professional**

In [ ]:
replacements = {
 'Mr.': 'Mr', 
 'Mrs.': 'Mrs/Miss', 
 'Miss.': 'Mrs/Miss', 
 'Master.': 'Master', 
 'Don.': 'Mr', 
 'Rev.': 'Officer/Professional', 
 'Dr.': 'Officer/Professional', 
 'Mme.': 'Mrs/Miss',
 'Ms.': 'Mrs/Miss', 
 'Major.': 'Officer/Professional', 
 'Lady.': 'Mrs/Miss', 
 'Sir.': 'Mr', 
 'Mlle.': 'Mrs/Miss', 
 'Col.': 'Officer/Professional', 
 'Capt.': 'Officer/Professional', 
 'the': 'Mrs/Miss',
 'Jonkheer.': 'Mr', 
 'Dona.': 'Mrs/Miss'
}

In [ ]:
all_data["Title"] = all_data["Title"].replace(replacements)

In [ ]:
all_data.head(15)

**I transform some of the categorical variables into one-hot encoded features. This makes it possible for the features to be fed to the models which require numerical input**

**when I use One-Hot Encoder,usually the features have low-dimenssions**

In [ ]:
one_hot_columns = ["Pclass", "Title", "Embarked", "Deck", "Sex"]

encoder = OneHotEncoder(sparse=False, dtype=int)
encoded_features = encoder.fit_transform(all_data[one_hot_columns])
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(one_hot_columns))
encoded_df.index = all_data.index

In [ ]:

df_final = pd.concat([all_data, encoded_df], axis=1)


**Drop Columns**

In [ ]:
df_final.head(10)

In [ ]:
drop_cols = ["Ticket", "Cabin", "Name", "PassengerId", "Age", "SibSp", "Parch", "Pclass", "Embarked", "Title", "Deck", "Sex"]

df_final.drop(columns=drop_cols, inplace=True)

In [ ]:
df_final.head(10)

In [ ]:
# Split the data back into the train and test sets
df_train = df_final[:891]
df_test = df_final[891:]

print(df_train.shape)
print(df_test.shape)

In [ ]:
df_test.drop(columns=['Survived'], inplace=True)

In [ ]:
X_train = df_train.drop(columns="Survived")
y_train = df_train["Survived"].values

X_test = df_test

# **Modeling**

**GridSearch**

In [ ]:
model = RandomForestClassifier(random_state=42,verbose=1,max_depth= 6, min_samples_leaf=5,min_samples_split=5,n_estimators=1500,oob_score=True)

In [ ]:
model.fit(X_train, y_train)
predictions = model.predict(X_test)



In [ ]:
X_train

In [ ]:
import joblib
joblib.dump(model, '/kaggle/working/random_forest_model.pkl')

pd.DataFrame(predictions, columns=['predictions']).to_csv('/kaggle/working/test_predictions.csv', index_label='id')

feature_importances_df =pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': model.feature_importances_
})

feature_importances_df = feature_importances_df.sort_values('Importance', ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(
    x= 'Importance',
    y= 'Feature',
    data=feature_importances_df.head(20)
)
plt.title('Top20_feature_importances')
plt.xlabel('Importance score')
plt.ylabel('Feature name')
plt.savefig('/kaggle/working/feature_importances.png', bbox_inches='tight')




**kaggle official submission**   

In [ ]:
kaggle_predictions = pd.DataFrame({
        "PassengerId": test_data["PassengerId"],
        "Survived": predictions.astype(int)
    })
def make_submission(kaggle_predictions):
    path="/kaggle/working/submission.csv"
    kaggle_predictions.to_csv(path, index=False)
    print(f"Submission exported to {path}")

make_submission(kaggle_predictions)